In [2]:
import pandas as pd

In [5]:
path = 'data/raw/participants/'

df = pd.read_csv(path+'train_candles.csv')

In [7]:
df['ticker'].unique()

array(['AFLT', 'ALRS', 'CHMF', 'GAZP', 'GMKN', 'LKOH', 'MAGN', 'MGNT',
       'MOEX', 'MTSS', 'NVTK', 'PHOR', 'PLZL', 'ROSN', 'RUAL', 'SBER',
       'SIBN', 'T', 'VTBR'], dtype=object)

In [ ]:
df.head()

,open,close,high,low,volume,begin,ticker,target_return_1d,target_direction_1d,target_return_20d,target_direction_20d
0,81.50,81.70,83.20,81.16,29755530,2020-06-19,AFLT,0.004896,1,0.071726,1
1,81.72,82.10,83.98,80.26,18502950,2020-06-22,AFLT,-0.010962,0,0.048477,1
2,82.04,81.20,82.48,80.40,16848930,2020-06-23,AFLT,-0.007635,0,0.048522,1
3,79.78,80.58,80.80,78.22,21559860,2020-06-25,AFLT,-0.014892,0,0.096550,1
4,80.50,79.38,81.44,78.76,14677280,2020-06-26,AFLT,0.044344,1,0.104056,1


In [ ]:
import pandas as pd

keywords = {
    'AFLT': ['Аэрофлот', 'Aeroflot', 'AFLT'],
    'ALRS': ['Алроса', 'Alrosa', 'ALRS'],
    'CHMF': ['Северсталь', 'Severstal', 'CHMF'],
    'GAZP': ['Газпром', 'Gazprom', 'GAZP'],
    'GMKN': ['Норильский никель', 'ГМК Норильский никель', 'Norilsk Nickel', 'GMKN'],
    'LKOH': ['Лукойл', 'Lukoil', 'LKOH'],
    'MAGN': ['ММК', 'Магнитогорский металлургический комбинат', 'Magnitogorsk', 'MAGN'],
    'MGNT': ['Магнит', 'Magnit', 'MGNT'],
    'MOEX': ['Московская биржа', 'MOEX'],
    'MTSS': ['МТС', 'MTS', 'MTSS'],
    'NVTK': ['Новатэк', 'Novatek', 'NVTK'],
    'PHOR': ['ФосАгро', 'PhosAgro', 'PHOR'],
    'PLZL': ['Полюс', 'Полюс Золото', 'Polyus', 'PLZL'],
    'ROSN': ['Роснефть', 'Rosneft', 'ROSN'],
    'RUAL': ['Русал', 'Rusal', 'RUAL'],
    'SBER': ['Сбербанк', 'Sberbank', 'SBER'],
    'SIBN': ['Сибур', 'Sibur', 'SIBN'],
    'T': ['Татнефть', 'Tatneft', 'T'],
    'VTBR': ['ВТБ', 'VTB', 'VTBR']
}

path = 'data/raw/participants/'

df_news = pd.read_csv(path+'train_news.csv')

def find_related_assets(text, keywords):
    """Ищет все активы, которые упоминаются в тексте"""
    related = []
    text_lower = text.lower()
    for ticker, kw_list in keywords.items():
        for kw in kw_list:
            if kw.lower() in text_lower:
                related.append(ticker)
                break
    return related

# Добавляем новую колонку с найденными активами
df_news['tickers'] = df_news['publication'].apply(lambda x: find_related_assets(x, keywords))

# Проверка
print(df_news[['publication', 'tickers']].head())


d = df_news['tickers'].isna()


num_empty = df_news['tickers'].apply(lambda x: len(x) == 0).sum()

# Количество новостей с хотя бы одним связанным активом
num_filled = df_news['tickers'].apply(lambda x: len(x) > 0).sum()

print(f"Пустых tickers: {num_empty}")
print(f"Заполненных tickers: {num_filled}")

                                         publication   related_assets
0  Тенденции в отрасли. Ключевые российские нефте...     [GAZP, LKOH]
1  Тенденции в отрасли. Ключевые российские стале...  [CHMF, MAGN, T]
2  Председатель правления "Газпрома" Алексей Милл...           [GAZP]
3  Дата и время ВОСА – 24.01.2020 23:59:59  Дата ...               []
4  Дата и время собрания - 27.12.2019 00:00:00  Д...               []
Пустых related_assets: 2376
Заполненных related_assets: 21751
